In [23]:
video_ids = range(1, 6)

In [24]:
# Load the JSON metadata for each video and store the paths, hits, and serves
import json

data = {}
for id in video_ids:
    with open(f'data/json/video{id}.json') as f:
        video_data = json.load(f)

        video_start = video_data['match']['start']
        video_end = video_data['match']['end']
        video_labels = {}

        # Initialize the labels for each frame
        for frame in range(video_start, video_end):
            video_labels[frame] = {}
            video_labels[frame]['shot']   = None    # Hit / Serve / Other
            video_labels[frame]['player'] = None    # Far / Near / Other
            video_labels[frame]['type']   = None    # Topspin / Slice / Flat / Other
            video_labels[frame]['side']   = None    # Forehand / Backhand / Smash / Other

        # Add the labels for each hit
        for hit in video_data['hits']:
            hit_start = int(hit['start'])
            hit_end = int(hit['end'])
            hit_label = hit['custom']

            for frame in range(hit_start, hit_end):
                video_labels[frame]['shot'] = 'Hit'
                video_labels[frame]['player'] = hit_label['Player']
                video_labels[frame]['type'] = hit_label['Type']
                video_labels[frame]['side'] = hit_label['Side']

        # Add the labels for each serve
        for serve in video_data['serves']:
            serve_start = int(serve['start'])
            serve_end = int(serve['end'])
            serve_label = serve['custom']

            for frame in range(serve_start, serve_end):
                video_labels[frame]['shot'] = 'Serve'
                video_labels[frame]['player'] = serve_label['Player']
                video_labels[frame]['type'] = serve_label['Result']
                video_labels[frame]['side'] = None

        data[id] = video_labels

{'shot': 'Serve', 'player': 'Near', 'type': 'In', 'side': None}
